In [1]:
# pip install -r requirements.txt --quiet

In [2]:
import pandas as pd
import data.building_ingestion_utils as biu
# import modeling_utils as mu
import data.data_utils as du
# import optimization_utils as ou
# import weather_ingestion_utils as wiu
import data.energy_price_utils as epu
import json


import warnings
warnings.filterwarnings('ignore')

directory = 'data/site_data/industrial_sites'

dataframes = du.restructure_dataframes(biu.get_dataframes(directory, ','))

In [3]:
print(f"Total dataframes created: {len(dataframes)}")

for name, df in dataframes.items():
    print(f'DataFrame: {name} loaded with shape: {df.shape}')

Total dataframes created: 3
DataFrame: industrial_site2 loaded with shape: (42760, 29)
DataFrame: industrial_site3 loaded with shape: (42760, 44)
DataFrame: industrial_site1 loaded with shape: (42760, 54)


Data is saved in dictionary of dataframes called `dataframe`

In [4]:
#   new_entry_key = 'energy'
#   new_entry_value = 'kWh'
# add_entry_to_units_dict(file_path, new_entry_key, new_entry_value, verbose=False)

units_dict = biu.read_units_dict_from_json('data/unit_dict.json')

dataframes = biu.add_units_to_column_names(dataframes, units_dict)

du.column_presence_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,General_Technical_Efficiency (%),General_Technical_Flow Rate (m3/h),General_Technical_Pressure (bar),General_Technical_Temperature (C),General_Vapour_Flow Rate (m3/h),General_Vapour_Fumes Temperature (C),General_Vapour_Pressure (bar),General_Vapour_Quantity (m3),General_Water_Flow Rate (m3/h),Time
0,industrial_site2,Column not present,Column not present,Column not present,float64,float64,float64,Column not present,Column not present,Column not present,...,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,object
1,industrial_site3,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,float64,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,object
2,industrial_site1,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,...,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,float64,float64,object


In [5]:
datetime_format = '%d/%m/%y %H:%M'
dataframes = du.convert_and_transform_date_columns(dict_of_dfs=dataframes, datetime_format=datetime_format, date_column='Time', verbose=True)
du.column_presence_checker(dataframes)



DataFrame: industrial_site2 - Converted Time to datetime
DataFrame: industrial_site3 - Converted Time to datetime
DataFrame: industrial_site1 - Converted Time to datetime


,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,Time,day_of_month_cos,day_of_month_sin,day_of_week_cos,day_of_week_sin,hour_cos,hour_sin,is_weekend,month_cos,month_sin
0,industrial_site2,Column not present,Column not present,Column not present,float64,float64,float64,Column not present,Column not present,Column not present,...,datetime64[ns],float64,float64,float64,float64,float64,float64,int64,float64,float64
1,industrial_site3,float64,float64,float64,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,datetime64[ns],float64,float64,float64,float64,float64,float64,int64,float64,float64
2,industrial_site1,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,float64,float64,float64,...,datetime64[ns],float64,float64,float64,float64,float64,float64,int64,float64,float64


# Create Target Column

## Remove end of data with no values

In [6]:
# Function to filter DataFrame
def filter_dataframe_by_date(df):
    df['Time'] = pd.to_datetime(df['Time'])
    return df[df['Time'] < '2023-07-01']

# Apply the function to each DataFrame in the dictionary
dataframes = {key: filter_dataframe_by_date(df) for key, df in dataframes.items()}



## Create Total Consumption (kW) Column

# Data Cleaning

## Imputing missing values with column averages by each dataframe

In [7]:
dataframes = du.impute_missing_values(dataframes, method='linear')

In [8]:
du.nan_checker(dataframes)

,DataFrame Name,01 General Transformer 1234_Electric_Active Energy (kWh),01 General Transformer 1234_Electric_Active Power (kW),01 General Transformer 1234_Electric_Power Factor (real),01 General_Electric_Active Energy (kWh),01 General_Electric_Active Power (kW),01 General_Electric_Power Factor (real),01 Normal General_Electric_Active Energy (kWh),01 Normal General_Electric_Active Power (kW),01 Normal General_Electric_Power Factor (real),...,Time,day_of_month_cos,day_of_month_sin,day_of_week_cos,day_of_week_sin,hour_cos,hour_sin,is_weekend,month_cos,month_sin
0,No NaNs,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
1,No NaNs,No NaNs,No NaNs,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs
2,No NaNs,Column not present,Column not present,Column not present,Column not present,Column not present,Column not present,No NaNs,No NaNs,No NaNs,...,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs,No NaNs


# Create Total Consumption (kWh) Column

In [9]:
# du.check_column_names(dataframes)
tempdf = dataframes['industrial_site1'].filter(['01 Normal General_Electric_Active Energy (kWh)','General_Electric_Active Energy (kWh)'], axis=1)
tempdf.head()


,01 Normal General_Electric_Active Energy (kWh),General_Electric_Active Energy (kWh)
0,441.0,123.0
1,412.0,117.0
2,440.0,123.0
3,448.0,126.0
4,410.0,123.0


# Drop Active Energy Columns
Data is not matching up with kW columns

In [10]:
du.drop_columns_with_keywords(dataframes, ['General_Electric', 'General Transformer 1234', 'Active Energy'])

In [11]:
dataframes = du.calculate_active_energy(dataframes)
du.check_column_names(dataframes)

DataFrame: industrial_site2
Columns: ['DataFrame Name', 'Time', '02 Production_Electric_Power Factor (real)', '03 Chiller Group_Electric_Power Factor (real)', '04 UTA_Electric_Power Factor (real)', '05 Compressors_Electric_Power Factor (real)', '06 Offices_Electric_Power Factor (real)', '07 Data Center_Electric_Power Factor (real)', '08 Technological Centers_Electric_Power Factor (real)', '02 Production_Electric_Active Power (kW)', '03 Chiller Group_Electric_Active Power (kW)', '04 UTA_Electric_Active Power (kW)', '05 Compressors_Electric_Active Power (kW)', '06 Offices_Electric_Active Power (kW)', '07 Data Center_Electric_Active Power (kW)', '08 Technological Centers_Electric_Active Power (kW)', 'is_weekend', 'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin', 'day_of_month_cos', '02 Production_Electric_Active Energy (kWh)', '03 Chiller Group_Electric_Active Energy (kWh)', '04 UTA_Electric_Active Energy (kWh)', '05 Compressors_El

In [12]:
dataframes = du.add_sum_column(dataframes, 'kWh', exclude_cols=['01 Normal General_Electric_Active Energy (kWh)','01 General Transformer 1234_Electric_Active Energy (kWh)', '01 General_Electric_Active Energy (kWh)', 'General_Electric_Active Energy (kWh)'])
du.preview_dict_of_dfs(dataframes)

Preview of DataFrame for industrial_site2:


,DataFrame Name,Time,02 Production_Electric_Power Factor (real),03 Chiller Group_Electric_Power Factor (real),04 UTA_Electric_Power Factor (real),05 Compressors_Electric_Power Factor (real),06 Offices_Electric_Power Factor (real),07 Data Center_Electric_Power Factor (real),08 Technological Centers_Electric_Power Factor (real),02 Production_Electric_Active Power (kW),...,day_of_month_sin,day_of_month_cos,02 Production_Electric_Active Energy (kWh),03 Chiller Group_Electric_Active Energy (kWh),04 UTA_Electric_Active Energy (kWh),05 Compressors_Electric_Active Energy (kWh),06 Offices_Electric_Active Energy (kWh),07 Data Center_Electric_Active Energy (kWh),08 Technological Centers_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site2,2022-07-01 00:00:00,0.49300,0.775,0.753,0.953,0.978,-0.77,0.005,65.7,...,0.201299,0.97953,16.425,52.0,10.100,4.175,0.735,0.650,8.675,92.760
1,industrial_site2,2022-07-01 00:15:00,-0.00480,0.787,0.757,0.951,0.978,-0.80,-0.910,58.9,...,0.201299,0.97953,14.725,51.5,10.125,3.875,0.755,0.650,7.625,89.255
2,industrial_site2,2022-07-01 00:30:00,-0.00400,0.780,0.759,0.951,-0.880,-0.77,-0.910,61.8,...,0.201299,0.97953,15.450,51.0,10.100,3.925,0.675,0.625,7.375,89.150
3,industrial_site2,2022-07-01 00:45:00,0.00025,0.793,0.758,0.955,-1.000,-0.77,-0.920,54.3,...,0.201299,0.97953,13.575,41.0,10.125,4.225,0.705,0.650,7.200,77.480
4,industrial_site2,2022-07-01 01:00:00,-0.00200,0.781,0.759,0.952,-0.900,-0.78,-0.910,54.5,...,0.201299,0.97953,13.625,51.5,10.125,4.525,0.810,0.650,7.625,88.860




Preview of DataFrame for industrial_site3:


,DataFrame Name,Time,General_Technical_Efficiency (%),02 Chiller Group_Electric_Power Factor (real),03 Aspirator_Electric_Power Factor (real),04 Compressed Air_Electric_Power Factor (real),05 Weaving_Electric_Power Factor (real),06 Ironing_Electric_Power Factor (real),07 UPS_Electric_Power Factor (real),08 General Services_Electric_Power Factor (real),...,05 Weaving_Electric_Active Energy (kWh),06 Ironing_Electric_Active Energy (kWh),07 UPS_Electric_Active Energy (kWh),08 General Services_Electric_Active Energy (kWh),09 UTA_Electric_Active Energy (kWh),10 Warehouses_Electric_Active Energy (kWh),11 Winding_Electric_Active Energy (kWh),12 Others_Electric_Active Energy (kWh),General_Technical_Active Energy (kWh),sum_of_kWh
0,industrial_site3,2022-07-01 00:00:00,0.0000,1.25,0.625,0.923,1.11,1.070,1.57,0.999,...,100.25,55.00,0.4950,8.425,18.900,2.70,0.9075,122.75,0.020,641.1975
1,industrial_site3,2022-07-01 00:15:00,4.2200,1.25,0.623,0.921,1.11,1.050,1.59,0.999,...,100.00,52.50,0.4950,8.000,18.900,2.65,0.7550,119.00,0.036,619.0860
2,industrial_site3,2022-07-01 00:30:00,1.9000,1.25,0.622,0.939,1.10,0.946,1.57,0.997,...,101.00,42.75,0.4975,8.275,19.075,2.65,0.7400,120.50,-0.009,606.7285
3,industrial_site3,2022-07-01 00:45:00,0.0000,1.25,0.623,0.942,1.11,1.110,1.59,1.010,...,97.00,42.50,0.4900,7.250,19.025,2.65,1.7500,119.25,-0.035,611.3800
4,industrial_site3,2022-07-01 01:00:00,0.0842,1.24,0.621,0.912,1.10,1.110,1.57,1.000,...,99.75,43.50,0.4975,7.925,18.975,2.65,1.6925,117.50,-0.300,611.1900




Preview of DataFrame for industrial_site1:


,DataFrame Name,Time,General_Natural Gas_Quantity (m3),General_Vapour_Quantity (m3),General_Vapour_Fumes Temperature (C),02 Chiller_Electric_Power Factor (real),03 Dyeing_Electric_Power Factor (real),04 Ironing_Electric_Power Factor (real),05 Purifier_Electric_Power Factor (real),07 Technological Centers_Electric_Power Factor (real),...,07 Technological Centers_Electric_Active Energy (kWh),08 Offices Changing Rooms_Electric_Active Energy (kWh),09 Compressed Air_Electric_Active Energy (kWh),10 General Services_Electric_Active Energy (kWh),11 UPS_Electric_Active Energy (kWh),12 UTA_Electric_Active Energy (kWh),14 Laboratory_Electric_Active Energy (kWh),15 Warehouses_Electric_Active Energy (kWh),17 Print_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site1,2022-07-01 00:00:00,0.0,0.0,179.0,0.872,0.907,1.13,0.930,0.834,...,27.000,17.150,43.75,10.050,1.6700,57.75,0.8500,3.325,17.325,365.995
1,industrial_site1,2022-07-01 00:15:00,0.0,0.0,186.0,0.370,0.905,1.14,0.925,0.839,...,23.625,17.825,39.00,9.650,1.7100,60.50,0.4700,3.325,26.500,362.505
2,industrial_site1,2022-07-01 00:30:00,0.0,0.0,187.0,0.370,0.914,1.14,0.703,0.862,...,34.000,16.775,43.75,9.625,1.6850,74.00,0.4600,3.325,26.000,420.495
3,industrial_site1,2022-07-01 00:45:00,0.0,4096.0,187.0,0.367,0.914,1.15,0.930,0.866,...,31.750,16.775,43.75,9.650,1.6700,58.50,0.4550,3.325,27.250,383.875
4,industrial_site1,2022-07-01 01:00:00,0.0,0.0,176.0,0.866,0.907,1.10,0.690,0.870,...,35.250,17.175,39.00,10.175,1.7125,58.75,0.7075,3.350,27.750,375.120


In [13]:
du.preview_dict_of_dfs(dataframes)

Preview of DataFrame for industrial_site2:


,DataFrame Name,Time,02 Production_Electric_Power Factor (real),03 Chiller Group_Electric_Power Factor (real),04 UTA_Electric_Power Factor (real),05 Compressors_Electric_Power Factor (real),06 Offices_Electric_Power Factor (real),07 Data Center_Electric_Power Factor (real),08 Technological Centers_Electric_Power Factor (real),02 Production_Electric_Active Power (kW),...,day_of_month_sin,day_of_month_cos,02 Production_Electric_Active Energy (kWh),03 Chiller Group_Electric_Active Energy (kWh),04 UTA_Electric_Active Energy (kWh),05 Compressors_Electric_Active Energy (kWh),06 Offices_Electric_Active Energy (kWh),07 Data Center_Electric_Active Energy (kWh),08 Technological Centers_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site2,2022-07-01 00:00:00,0.49300,0.775,0.753,0.953,0.978,-0.77,0.005,65.7,...,0.201299,0.97953,16.425,52.0,10.100,4.175,0.735,0.650,8.675,92.760
1,industrial_site2,2022-07-01 00:15:00,-0.00480,0.787,0.757,0.951,0.978,-0.80,-0.910,58.9,...,0.201299,0.97953,14.725,51.5,10.125,3.875,0.755,0.650,7.625,89.255
2,industrial_site2,2022-07-01 00:30:00,-0.00400,0.780,0.759,0.951,-0.880,-0.77,-0.910,61.8,...,0.201299,0.97953,15.450,51.0,10.100,3.925,0.675,0.625,7.375,89.150
3,industrial_site2,2022-07-01 00:45:00,0.00025,0.793,0.758,0.955,-1.000,-0.77,-0.920,54.3,...,0.201299,0.97953,13.575,41.0,10.125,4.225,0.705,0.650,7.200,77.480
4,industrial_site2,2022-07-01 01:00:00,-0.00200,0.781,0.759,0.952,-0.900,-0.78,-0.910,54.5,...,0.201299,0.97953,13.625,51.5,10.125,4.525,0.810,0.650,7.625,88.860




Preview of DataFrame for industrial_site3:


,DataFrame Name,Time,General_Technical_Efficiency (%),02 Chiller Group_Electric_Power Factor (real),03 Aspirator_Electric_Power Factor (real),04 Compressed Air_Electric_Power Factor (real),05 Weaving_Electric_Power Factor (real),06 Ironing_Electric_Power Factor (real),07 UPS_Electric_Power Factor (real),08 General Services_Electric_Power Factor (real),...,05 Weaving_Electric_Active Energy (kWh),06 Ironing_Electric_Active Energy (kWh),07 UPS_Electric_Active Energy (kWh),08 General Services_Electric_Active Energy (kWh),09 UTA_Electric_Active Energy (kWh),10 Warehouses_Electric_Active Energy (kWh),11 Winding_Electric_Active Energy (kWh),12 Others_Electric_Active Energy (kWh),General_Technical_Active Energy (kWh),sum_of_kWh
0,industrial_site3,2022-07-01 00:00:00,0.0000,1.25,0.625,0.923,1.11,1.070,1.57,0.999,...,100.25,55.00,0.4950,8.425,18.900,2.70,0.9075,122.75,0.020,641.1975
1,industrial_site3,2022-07-01 00:15:00,4.2200,1.25,0.623,0.921,1.11,1.050,1.59,0.999,...,100.00,52.50,0.4950,8.000,18.900,2.65,0.7550,119.00,0.036,619.0860
2,industrial_site3,2022-07-01 00:30:00,1.9000,1.25,0.622,0.939,1.10,0.946,1.57,0.997,...,101.00,42.75,0.4975,8.275,19.075,2.65,0.7400,120.50,-0.009,606.7285
3,industrial_site3,2022-07-01 00:45:00,0.0000,1.25,0.623,0.942,1.11,1.110,1.59,1.010,...,97.00,42.50,0.4900,7.250,19.025,2.65,1.7500,119.25,-0.035,611.3800
4,industrial_site3,2022-07-01 01:00:00,0.0842,1.24,0.621,0.912,1.10,1.110,1.57,1.000,...,99.75,43.50,0.4975,7.925,18.975,2.65,1.6925,117.50,-0.300,611.1900




Preview of DataFrame for industrial_site1:


,DataFrame Name,Time,General_Natural Gas_Quantity (m3),General_Vapour_Quantity (m3),General_Vapour_Fumes Temperature (C),02 Chiller_Electric_Power Factor (real),03 Dyeing_Electric_Power Factor (real),04 Ironing_Electric_Power Factor (real),05 Purifier_Electric_Power Factor (real),07 Technological Centers_Electric_Power Factor (real),...,07 Technological Centers_Electric_Active Energy (kWh),08 Offices Changing Rooms_Electric_Active Energy (kWh),09 Compressed Air_Electric_Active Energy (kWh),10 General Services_Electric_Active Energy (kWh),11 UPS_Electric_Active Energy (kWh),12 UTA_Electric_Active Energy (kWh),14 Laboratory_Electric_Active Energy (kWh),15 Warehouses_Electric_Active Energy (kWh),17 Print_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site1,2022-07-01 00:00:00,0.0,0.0,179.0,0.872,0.907,1.13,0.930,0.834,...,27.000,17.150,43.75,10.050,1.6700,57.75,0.8500,3.325,17.325,365.995
1,industrial_site1,2022-07-01 00:15:00,0.0,0.0,186.0,0.370,0.905,1.14,0.925,0.839,...,23.625,17.825,39.00,9.650,1.7100,60.50,0.4700,3.325,26.500,362.505
2,industrial_site1,2022-07-01 00:30:00,0.0,0.0,187.0,0.370,0.914,1.14,0.703,0.862,...,34.000,16.775,43.75,9.625,1.6850,74.00,0.4600,3.325,26.000,420.495
3,industrial_site1,2022-07-01 00:45:00,0.0,4096.0,187.0,0.367,0.914,1.15,0.930,0.866,...,31.750,16.775,43.75,9.650,1.6700,58.50,0.4550,3.325,27.250,383.875
4,industrial_site1,2022-07-01 01:00:00,0.0,0.0,176.0,0.866,0.907,1.10,0.690,0.870,...,35.250,17.175,39.00,10.175,1.7125,58.75,0.7075,3.350,27.750,375.120


# Clean Power Factor

In [14]:
dataframes = du.clean_data_by_column(dataframes, column_suffix='_Power Factor (real)',
                         negative_value_strategy='abs', outlier_strategy='average',
                         missing_value_strategy='average')
du.preview_dict_of_dfs(dataframes)

Preview of DataFrame for industrial_site2:


,DataFrame Name,Time,02 Production_Electric_Power Factor (real),03 Chiller Group_Electric_Power Factor (real),04 UTA_Electric_Power Factor (real),05 Compressors_Electric_Power Factor (real),06 Offices_Electric_Power Factor (real),07 Data Center_Electric_Power Factor (real),08 Technological Centers_Electric_Power Factor (real),02 Production_Electric_Active Power (kW),...,day_of_month_sin,day_of_month_cos,02 Production_Electric_Active Energy (kWh),03 Chiller Group_Electric_Active Energy (kWh),04 UTA_Electric_Active Energy (kWh),05 Compressors_Electric_Active Energy (kWh),06 Offices_Electric_Active Energy (kWh),07 Data Center_Electric_Active Energy (kWh),08 Technological Centers_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site2,2022-07-01 00:00:00,0.49300,0.775,0.753,0.953,0.978,0.77,0.005,65.7,...,0.201299,0.97953,16.425,52.0,10.100,4.175,0.735,0.650,8.675,92.760
1,industrial_site2,2022-07-01 00:15:00,0.00480,0.787,0.757,0.951,0.978,0.80,0.910,58.9,...,0.201299,0.97953,14.725,51.5,10.125,3.875,0.755,0.650,7.625,89.255
2,industrial_site2,2022-07-01 00:30:00,0.00400,0.780,0.759,0.951,0.880,0.77,0.910,61.8,...,0.201299,0.97953,15.450,51.0,10.100,3.925,0.675,0.625,7.375,89.150
3,industrial_site2,2022-07-01 00:45:00,0.00025,0.793,0.758,0.955,1.000,0.77,0.920,54.3,...,0.201299,0.97953,13.575,41.0,10.125,4.225,0.705,0.650,7.200,77.480
4,industrial_site2,2022-07-01 01:00:00,0.00200,0.781,0.759,0.952,0.900,0.78,0.910,54.5,...,0.201299,0.97953,13.625,51.5,10.125,4.525,0.810,0.650,7.625,88.860




Preview of DataFrame for industrial_site3:


,DataFrame Name,Time,General_Technical_Efficiency (%),02 Chiller Group_Electric_Power Factor (real),03 Aspirator_Electric_Power Factor (real),04 Compressed Air_Electric_Power Factor (real),05 Weaving_Electric_Power Factor (real),06 Ironing_Electric_Power Factor (real),07 UPS_Electric_Power Factor (real),08 General Services_Electric_Power Factor (real),...,05 Weaving_Electric_Active Energy (kWh),06 Ironing_Electric_Active Energy (kWh),07 UPS_Electric_Active Energy (kWh),08 General Services_Electric_Active Energy (kWh),09 UTA_Electric_Active Energy (kWh),10 Warehouses_Electric_Active Energy (kWh),11 Winding_Electric_Active Energy (kWh),12 Others_Electric_Active Energy (kWh),General_Technical_Active Energy (kWh),sum_of_kWh
0,industrial_site3,2022-07-01 00:00:00,0.0000,1.25,0.625,0.923,1.11,1.070,1.57,0.999,...,100.25,55.00,0.4950,8.425,18.900,2.70,0.9075,122.75,0.020,641.1975
1,industrial_site3,2022-07-01 00:15:00,4.2200,1.25,0.623,0.921,1.11,1.050,1.59,0.999,...,100.00,52.50,0.4950,8.000,18.900,2.65,0.7550,119.00,0.036,619.0860
2,industrial_site3,2022-07-01 00:30:00,1.9000,1.25,0.622,0.939,1.10,0.946,1.57,0.997,...,101.00,42.75,0.4975,8.275,19.075,2.65,0.7400,120.50,-0.009,606.7285
3,industrial_site3,2022-07-01 00:45:00,0.0000,1.25,0.623,0.942,1.11,1.110,1.59,1.010,...,97.00,42.50,0.4900,7.250,19.025,2.65,1.7500,119.25,-0.035,611.3800
4,industrial_site3,2022-07-01 01:00:00,0.0842,1.24,0.621,0.912,1.10,1.110,1.57,1.000,...,99.75,43.50,0.4975,7.925,18.975,2.65,1.6925,117.50,-0.300,611.1900




Preview of DataFrame for industrial_site1:


,DataFrame Name,Time,General_Natural Gas_Quantity (m3),General_Vapour_Quantity (m3),General_Vapour_Fumes Temperature (C),02 Chiller_Electric_Power Factor (real),03 Dyeing_Electric_Power Factor (real),04 Ironing_Electric_Power Factor (real),05 Purifier_Electric_Power Factor (real),07 Technological Centers_Electric_Power Factor (real),...,07 Technological Centers_Electric_Active Energy (kWh),08 Offices Changing Rooms_Electric_Active Energy (kWh),09 Compressed Air_Electric_Active Energy (kWh),10 General Services_Electric_Active Energy (kWh),11 UPS_Electric_Active Energy (kWh),12 UTA_Electric_Active Energy (kWh),14 Laboratory_Electric_Active Energy (kWh),15 Warehouses_Electric_Active Energy (kWh),17 Print_Electric_Active Energy (kWh),sum_of_kWh
0,industrial_site1,2022-07-01 00:00:00,0.0,0.0,179.0,0.872,0.907,1.13,0.930,0.834,...,27.000,17.150,43.75,10.050,1.6700,57.75,0.8500,3.325,17.325,365.995
1,industrial_site1,2022-07-01 00:15:00,0.0,0.0,186.0,0.370,0.905,1.14,0.925,0.839,...,23.625,17.825,39.00,9.650,1.7100,60.50,0.4700,3.325,26.500,362.505
2,industrial_site1,2022-07-01 00:30:00,0.0,0.0,187.0,0.370,0.914,1.14,0.703,0.862,...,34.000,16.775,43.75,9.625,1.6850,74.00,0.4600,3.325,26.000,420.495
3,industrial_site1,2022-07-01 00:45:00,0.0,4096.0,187.0,0.367,0.914,1.15,0.930,0.866,...,31.750,16.775,43.75,9.650,1.6700,58.50,0.4550,3.325,27.250,383.875
4,industrial_site1,2022-07-01 01:00:00,0.0,0.0,176.0,0.866,0.907,1.10,0.690,0.870,...,35.250,17.175,39.00,10.175,1.7125,58.75,0.7075,3.350,27.750,375.120


In [15]:
def adjust_outliers_via_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    def adjust_value(x):
        if x < lower_bound:
            return lower_bound
        elif x > upper_bound:
            return upper_bound
        else:
            return x

    df[column] = df[column].apply(adjust_value)
    
# for name, df in dataframes.items():
#     # Find columns with the suffix "_Power Factor (real)"
#     power_factor_columns = [col for col in df.columns if col.endswith('_Power Factor (real)')]
#     for column in power_factor_columns:
#         # Adjust outliers in the column
#         adjust_outliers_via_iqr(df, column)
#         
# du.preview_dict_of_dfs(dataframes)

# Add Apparent Power Column

In [16]:
dataframes = du.calculate_apparent_power(dataframes)

# Aggregating Data
15 minute data -> hourly data

In [17]:
dataframes = du.aggregate_to_hourly(dataframes, 'Time')
du.preview_dict_of_dfs(dataframes)

Preview of DataFrame for industrial_site2:


,Time,02 Production_Electric_Active Energy (kWh),02 Production_Electric_Active Power (kW),02 Production_Electric_Apparent Power (kVa),02 Production_Electric_Power Factor (real),03 Chiller Group_Electric_Active Energy (kWh),03 Chiller Group_Electric_Active Power (kW),03 Chiller Group_Electric_Apparent Power (kVa),03 Chiller Group_Electric_Power Factor (real),04 UTA_Electric_Active Energy (kWh),...,DataFrame Name,is_weekend,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,month_sin,month_cos,day_of_week_sin,day_of_week_cos
Time,,,,,,,,,,,,,,,,,,,,,
0,2022-07-01 00:00:00,60.175,60.175,61263.524763,0.125512,195.50,195.50,249.622159,0.78375,40.450,...,industrial_site2,0.0,0.000000,1.000000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
1,2022-07-01 01:00:00,55.475,55.475,16023.672161,0.004325,195.25,195.25,250.552254,0.77975,40.500,...,industrial_site2,0.0,0.258819,0.965926,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
2,2022-07-01 02:00:00,62.200,62.200,38621.739445,0.002225,162.25,162.25,204.214807,0.79450,40.525,...,industrial_site2,0.0,0.500000,0.866025,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
3,2022-07-01 03:00:00,56.175,56.175,44397.222222,0.001888,157.50,157.50,201.728232,0.78075,40.550,...,industrial_site2,0.0,0.707107,0.707107,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
4,2022-07-01 04:00:00,57.350,57.350,40666.346270,0.249750,158.25,158.25,201.656297,0.78475,40.575,...,industrial_site2,0.0,0.866025,0.500000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969




Preview of DataFrame for industrial_site3:


,Time,02 Chiller Group_Electric_Active Energy (kWh),02 Chiller Group_Electric_Active Power (kW),02 Chiller Group_Electric_Apparent Power (kVa),02 Chiller Group_Electric_Power Factor (real),03 Aspirator_Electric_Active Energy (kWh),03 Aspirator_Electric_Active Power (kW),03 Aspirator_Electric_Apparent Power (kVa),03 Aspirator_Electric_Power Factor (real),04 Compressed Air_Electric_Active Energy (kWh),...,DataFrame Name,is_weekend,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,month_sin,month_cos,day_of_week_sin,day_of_week_cos
Time,,,,,,,,,,,,,,,,,,,,,
0,2022-07-01 00:00:00,329.00,329.00,263.200000,1.25000,418.50,418.50,671.482514,0.62325,533.75,...,industrial_site3,0.0,0.000000,1.000000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
1,2022-07-01 01:00:00,342.25,342.25,276.020598,1.24000,422.75,422.75,678.021689,0.62350,555.25,...,industrial_site3,0.0,0.258819,0.965926,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
2,2022-07-01 02:00:00,338.25,338.25,275.026869,1.23000,424.75,424.75,680.964516,0.62375,556.25,...,industrial_site3,0.0,0.500000,0.866025,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
3,2022-07-01 03:00:00,337.00,337.00,406.548579,1.02925,416.25,416.25,668.673517,0.62250,524.50,...,industrial_site3,0.0,0.707107,0.707107,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
4,2022-07-01 04:00:00,338.75,338.75,275.432972,1.23000,422.75,422.75,678.018627,0.62350,565.00,...,industrial_site3,0.0,0.866025,0.500000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969




Preview of DataFrame for industrial_site1:


,Time,02 Chiller_Electric_Active Energy (kWh),02 Chiller_Electric_Active Power (kW),02 Chiller_Electric_Apparent Power (kVa),02 Chiller_Electric_Power Factor (real),03 Dyeing_Electric_Active Energy (kWh),03 Dyeing_Electric_Active Power (kW),03 Dyeing_Electric_Apparent Power (kVa),03 Dyeing_Electric_Power Factor (real),04 Ironing_Electric_Active Energy (kWh),...,DataFrame Name,is_weekend,hour_sin,hour_cos,day_of_month_sin,day_of_month_cos,month_sin,month_cos,day_of_week_sin,day_of_week_cos
Time,,,,,,,,,,,,,,,,,,,,,
0,2022-07-01 00:00:00,238.25,238.25,552.140570,0.49475,241.50,241.50,265.294197,0.91000,191.25,...,industrial_site1,0.0,0.000000,1.000000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
1,2022-07-01 01:00:00,218.75,218.75,252.748623,0.86550,243.25,243.25,268.181243,0.90700,166.75,...,industrial_site1,0.0,0.258819,0.965926,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
2,2022-07-01 02:00:00,222.75,222.75,430.496768,0.61725,214.50,214.50,239.897754,0.89425,195.50,...,industrial_site1,0.0,0.500000,0.866025,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
3,2022-07-01 03:00:00,212.50,212.50,319.192633,0.75675,230.75,230.75,256.744323,0.89875,166.00,...,industrial_site1,0.0,0.707107,0.707107,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969
4,2022-07-01 04:00:00,217.50,217.50,326.892961,0.75825,240.00,240.00,263.603414,0.91050,164.50,...,industrial_site1,0.0,0.866025,0.500000,0.201299,0.97953,-0.5,-0.866025,-0.433884,-0.900969


# Ingest Price Data

In [18]:
pd.DataFrame(du.get_min_max_dates(dataframes))

,industrial_site2,industrial_site3,industrial_site1
earliest_date,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00
latest_date,2023-06-30 23:00:00,2023-06-30 23:00:00,2023-06-30 23:00:00


In [19]:
import os
import re
from datetime import datetime, timedelta

# Define the directory containing the files
directory = 'data/energy_price_data/marginalpdbc'

# Regex pattern to match the filenames and extract dates
pattern = re.compile(r'marginalpdbc_(\d{8})\.\d')

# Get a list of files in the directory
files = os.listdir(directory)

# Extract dates from filenames
dates = []
for file in files:
    match = pattern.match(file)
    if match:
        date_str = match.group(1)
        date = datetime.strptime(date_str, '%Y%m%d')
        dates.append(date)

# Sort dates
dates.sort()

# Find the range of dates
if dates:
    start_date = dates[0]
    end_date = dates[-1]

    # Generate the complete date range
    complete_range = set(start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1))

    # Find missing dates
    missing_dates = complete_range - set(dates)

    # Print missing dates
    if missing_dates:
        print("Missing dates:")
        for missing_date in sorted(missing_dates):
            print(missing_date.strftime('%Y%m%d'))
    else:
        print("No missing dates.")
else:
    print("No valid files found in the directory.")

No missing dates.


In [20]:
csv_data = pd.read_csv('data/energy_price_data/spain_daily_market_hourly_prices/marginalpdbc_2022.csv')
csv_data.head()

,Date,Price1,Price2,Year
0,2022-03-18 00:00:00,203.00,203.00,2022
1,2022-03-18 01:00:00,199.00,199.00,2022
2,2022-03-18 02:00:00,198.88,198.88,2022
3,2022-03-18 03:00:00,198.88,198.88,2022
4,2022-03-18 04:00:00,198.88,198.88,2022


In [21]:
dataframes = epu.merge_price_data(dataframes, 'data/energy_price_data/spain_daily_market_hourly_prices/', 'Time', 'Date', 'Price1')


In [22]:
du.add_energy_cost_column(dataframes, 'price_mWh', 'sum_of_kWh')

In [23]:
du.check_column_names(dataframes)

DataFrame: industrial_site2
Columns: ['Time', '02 Production_Electric_Active Energy (kWh)', '02 Production_Electric_Active Power (kW)', '02 Production_Electric_Apparent Power (kVa)', '02 Production_Electric_Power Factor (real)', '03 Chiller Group_Electric_Active Energy (kWh)', '03 Chiller Group_Electric_Active Power (kW)', '03 Chiller Group_Electric_Apparent Power (kVa)', '03 Chiller Group_Electric_Power Factor (real)', '04 UTA_Electric_Active Energy (kWh)', '04 UTA_Electric_Active Power (kW)', '04 UTA_Electric_Apparent Power (kVa)', '04 UTA_Electric_Power Factor (real)', '05 Compressors_Electric_Active Energy (kWh)', '05 Compressors_Electric_Active Power (kW)', '05 Compressors_Electric_Apparent Power (kVa)', '05 Compressors_Electric_Power Factor (real)', '06 Offices_Electric_Active Energy (kWh)', '06 Offices_Electric_Active Power (kW)', '06 Offices_Electric_Apparent Power (kVa)', '06 Offices_Electric_Power Factor (real)', '07 Data Center_Electric_Active Energy (kWh)', '07 Data Center_

# Join Weather Data

In [24]:
from data.weather_data.weather_ingestion_utils import calculate_heat_index

weather_data = pd.read_csv('data/weather_data/processed_data/madrid,spain 2022-07-01 to 2023-09-20.csv')

weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

weather_data = calculate_heat_index(weather_data)

for name, df in dataframes.items():
    df['datetime_hourly'] = df['Time'].dt.floor('h')
    merged_df = pd.merge(df, weather_data, left_on=df['datetime_hourly'] , right_on=weather_data['datetime'], how='left')
    
    merged_df.drop(columns=['key_0', 'datetime', 'datetime_hourly'], inplace=True)
    
    dataframes[name] = merged_df
    

    

# Save data to CSVs

In [25]:
du.save_dataframes_to_csv(dataframes, 'data/processed_data/industrial_sites_processed', verbose=True)

Dataframe 'industrial_site2' saved to 'data/processed_data/industrial_sites_processed/2024-07-09T19-25-47_industrial_site2_processed.csv'
Dataframe 'industrial_site3' saved to 'data/processed_data/industrial_sites_processed/2024-07-09T19-25-47_industrial_site3_processed.csv'
Dataframe 'industrial_site1' saved to 'data/processed_data/industrial_sites_processed/2024-07-09T19-25-47_industrial_site1_processed.csv'
